In [120]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import openpyxl
import geopandas as gpd

In [118]:
input_path='../../data/input/dhis2/new_system/'
output_path='../../data/output/sprint3_analysis/'
shapes_path='../../data/shapes/district/districts_17_19.shp'

In [119]:
df_Jan = pd.read_csv(input_path+'Data_elements_(Cases)_jan.csv')
df_Feb = pd.read_csv(input_path+'Data_elements_(Cases)_feb.csv')
df_Mar = pd.read_csv(input_path+'Data_elements_(Cases)_march.csv')
df_Apr = pd.read_csv(input_path+'Data_elements_(Cases)_april.csv')

In [10]:
districts = df_Feb[df_Feb['Data']=='033B-CD01a. Malaria (diagnosed)  - Cases']['Organisation unit']

In [11]:
def data_Quality(indicator):
    df = pd.DataFrame(districts).reset_index(drop=True)

    Jan = df_Jan[df_Jan['Data']==indicator].drop('Data', axis=1)
    Feb = df_Feb[df_Feb['Data']==indicator].drop('Data', axis=1)
    March = df_Mar[df_Mar['Data']==indicator].drop('Data', axis=1)
    April = df_Apr[df_Apr['Data']==indicator].drop('Data', axis=1)
    
    #print(Jan.shape[0]+ Feb.shape[0]+ March.shape[0] + April.shape[0])

    df_final = Jan.merge(Feb,on='Organisation unit', how='outer').merge(March,on='Organisation unit', how='outer').merge(April,on='Organisation unit', how='outer')
    #df_final.columns = ["Districts", "Jan_"+indicator, "Feb_"+indicator, "Mar_"+indicator, "Apr_"+indicator]
    df_final.set_index('Organisation unit',drop=True, inplace=True)
    columns = [(indicator,"Jan"), (indicator,"Feb"),(indicator,"Mar"),(indicator,"Avr")]
    df_final.columns=pd.MultiIndex.from_tuples(columns)
    return df_final


In [12]:
data_Quality('033B-CD01a. Malaria (diagnosed)  - Cases')

033B-CD01a. Malaria (diagnosed)  - Cases                   \
                                                         Jan      Feb     Mar   
Organisation unit                                                               
Amuria District                                       2025.0   6710.0  7877.0   
Amuru District                                        5622.0   9317.0  8246.0   
Sironko District                                       398.0   1964.0  1868.0   
Kitgum District                                       1944.0   6135.0  6420.0   
Kitagwenda District                                    537.0   2732.0  1926.0   
...                                                      ...      ...     ...   
Karenga District                                        46.0   1614.0  1607.0   
Mayuge District                                       2625.0  12442.0  9096.0   
Moroto District                                       1285.0   3315.0  1734.0   
Sembabule District                                    3671.0   4672.0  2309.0   
Amudat District                                          NaN   1328.0   568.0   

                              
                         Avr  
Organisation unit             
Amuria District       9388.0  
Amuru District       10237.0  
Sironko District      1841.0  
Kitgum District       7846.0  
Kitagwenda District   2765.0  
...                      ...  
Karenga District      1470.0  
Mayuge District       7701.0  
Moroto District       2439.0  
Sembabule District    2545.0  
Amudat District       1388.0  

[135 rows x 4 columns]

# Data elements

In [17]:
multi_index=pd.MultiIndex.from_tuples([('Test','test')])
data_elements=pd.DataFrame(index=districts,columns=multi_index)

In [18]:
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = data_Quality(indicator)
    data_elements = pd.merge(data_elements, df_months, how='left',left_index=True,right_index=True) 

In [19]:
data_elements.drop('test',axis=1,inplace=True,level=1)

In [20]:
null_sum1= data_elements.count()/135

# Indicators 

In [21]:
df_Jan = pd.read_csv(input_path+'Indicator_January.csv')
df_Feb = pd.read_csv(input_path+'Indicator_Feb.csv')
df_Mar = pd.read_csv(input_path+'Indicator_march.csv')
df_Apr = pd.read_csv(input_path+'Indicator_April.csv')

In [22]:
districts = df_Feb[df_Feb['Data']=='EPI - BCG doses given']['Organisation unit'].unique()
multi_index=pd.MultiIndex.from_tuples([('Test','test')])

In [23]:
data_Indicators=pd.DataFrame(index=districts,columns=multi_index)

In [24]:
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = data_Quality(indicator)
    data_Indicators = pd.merge(data_Indicators, df_months, how='left',left_index=True,right_index=True)

In [25]:
data_Indicators.reset_index(inplace=True)
data_Indicators=data_Indicators.groupby('index',axis=0).mean() # This shouldnt be needed, there might be a bug

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [71]:
null_sum2= data_Indicators.count()/135

105- Total Linked to HIV care (1-4 Yrs) - Female                          Jan    0.644444
                                                                          Feb    0.644444
                                                                          Mar    0.644444
                                                                          Avr    0.562963
105- Total Linked to HIV care (1-4 Yrs) - Male                            Jan    0.651852
                                                                                   ...   
TB-Weekly - Percentage  of samples tested with GeneXpert out of expected  Avr    0.814815
TB-Weekly - Percentage  of weekly attendance screened for TB              Jan    0.681481
                                                                          Feb    1.000000
                                                                          Mar    1.000000
                                                                          Avr    1.000000
Length: 10

# Putting the data together and making sure teh district names match our shapefile

In [27]:
# Looking at data quality and null data in one place

null_sum=pd.concat([null_sum1,null_sum2])
null_sum_df = pd.DataFrame(null_sum)
null_sum_df.columns = ['% of data available']
null_sum_df.index.rename(['indicator','month'],inplace=True)
null_av_df=null_sum_df.groupby('indicator')['% of data available'].mean()

In [28]:
#Putting both our data together
data=pd.merge(data_Indicators,data_elements,how='inner',left_index=True,right_index=True)

In [124]:
data.rename(index=lambda s: s [:-9],inplace=True)
data.rename(index=str.upper,inplace=True)

In [126]:
data.head()

105- Total Linked to HIV care (1-4 Yrs) - Female                 \
                                                      Jan  Feb  Mar  Avr   
ABIM                                                  NaN  0.0  0.0  2.0   
ADJUMANI                                              1.0  1.0  NaN  0.0   
AGAGO                                                 0.0  0.0  0.0  1.0   
ALEBTONG                                              NaN  NaN  1.0  NaN   
AMOLATAR                                              0.0  NaN  1.0  NaN   

         105- Total Linked to HIV care (1-4 Yrs) - Male                 \
                                                    Jan  Feb  Mar  Avr   
ABIM                                                NaN  0.0  0.0  NaN   
ADJUMANI                                            1.0  0.0  3.0  1.0   
AGAGO                                               0.0  2.0  1.0  0.0   
ALEBTONG                                            1.0  NaN  NaN  NaN   
AMOLATAR                                            2.0  NaN  NaN  1.0   

         105- Total Linked to HIV care (10-14Yrs) - Female       ...  \
                                                       Jan  Feb  ...   
ABIM                                                   NaN  NaN  ...   
ADJUMANI                                               0.0  0.0  ...   
AGAGO                                                  0.0  1.0  ...   
ALEBTONG                                               0.0  1.0  ...   
AMOLATAR                                               0.0  NaN  ...   

         033B-CD14d. Hepatitis B- Cases Positive      \
                                             Mar Avr   
ABIM                                         NaN NaN   
ADJUMANI                                     NaN NaN   
AGAGO                                        NaN NaN   
ALEBTONG                                     NaN NaN   
AMOLATAR                                     NaN NaN   

         033B-CD13d. Typhoid Fever- Cases Positive              \
                                               Jan Feb Mar Avr   
ABIM                                           NaN NaN NaN NaN   
ADJUMANI                                       NaN NaN NaN NaN   
AGAGO                                          NaN NaN NaN NaN   
ALEBTONG                                       NaN NaN NaN NaN   
AMOLATAR                                       NaN NaN NaN NaN   

         033B-HI11b. Adverse Drug Reactions (ADR) - Deaths              
                                                       Jan Feb Mar Avr  
ABIM                                                   NaN NaN NaN NaN  
ADJUMANI                                               NaN NaN NaN NaN  
AGAGO                                                  NaN NaN NaN NaN  
ALEBTONG                                               NaN NaN NaN NaN  
AMOLATAR                                               NaN NaN NaN NaN  

[5 rows x 1444 columns]

In [121]:
shapes = gpd.read_file(shapes_path)

In [150]:
list(set(shapes['name_19']).difference(set(data_target.index)))

['SSEMBABULE', 'MADI OKOLLO']

In [151]:
list(set(data_target.index).difference(set(shapes['name_19'])))

['SEMBABULE', 'MADI-OKOLLO']

In [ ]:
dictionary = dict(zip(keys, values))

# Selecting relevant indicators

In [127]:
target = pd.read_excel(input_path+'Data summary.xlsx',sheet_name='target')

In [128]:
target = pd.merge(target,null_av_df,left_on='Var',right_on='indicator',how='left')
target['keep']=(target['% of data available']>0.7)

In [129]:
with ExcelWriter(input_path+'Data summary.xlsx',mode='a') as writer:
   target.to_excel(writer,sheet_name='target_availability')

In [130]:
target_indicators=list(target.loc[target['keep']==1,'Var'])


# Look at EPI data

In [131]:
epi_target=['EPI - BCG doses given','EPI - BCG to MR 1 dropout rate (%)','EPI - DPT 1 to DPT3 dropout rate (%)']

In [141]:
data_target=data.iloc[:, data.columns.get_level_values(0).isin(epi_target)].copy()
#data_target.reset_index(inplace=True)

In [133]:
# Is the delta over 4 month the best measure?

print(null_sum.loc[epi_target,['Jan','Feb','Mar','Avr']])

# Loos like yes, as the reporting rates seem steady over the period 


indicator                             month
EPI - BCG doses given                 Jan      1.000000
                                      Feb      1.000000
                                      Mar      1.000000
                                      Avr      1.000000
EPI - BCG to MR 1 dropout rate (%)    Jan      1.000000
                                      Feb      1.000000
                                      Mar      1.000000
                                      Avr      0.992593
EPI - DPT 1 to DPT3 dropout rate (%)  Jan      1.000000
                                      Feb      1.000000
                                      Mar      1.000000
                                      Avr      0.992593
dtype: float64


In [142]:
def get_delta(indicator):
    delta=((data_target[(indicator,'Avr')]
            -data_target[(indicator,'Jan')])
           /data_target[(indicator,'Jan')])
    return delta

In [143]:
for indicator in epi_target:
    data_target[(indicator,'delta_4')]=get_delta(indicator)

Weird phenomenon : negative drop out rates 

"Check for inconsistency in related indicators – **An example of such inconsistency is when there is a negative dropout over a full year** between the number of children receiving the first dose of DPT vaccine and the number receiving the third dose of the vaccine.  Another example, as illustrated in Figure 1 above, is inconsistency between the number of children receiving DPT vaccine and the number of children receiving OPV vaccine.  Yet another example is when there is a negative “wastage rate” (discussed below) -- when the number of children reported to have received a vaccine is greater than the number of doses of the vaccine reported to have been consumed."

from this [WHO document](https://www.who.int/healthinfo/FacilityAnalysisGuide_Immunization.pdf?ua=1#page=11)


In [144]:
data_target.head()

EPI - BCG doses given                        \
                           Jan     Feb    Mar    Avr   
ABIM                     354.0   294.0  342.0  310.0   
ADJUMANI                 976.0  1055.0  866.0  864.0   
AGAGO                    736.0   667.0  670.0  510.0   
ALEBTONG                 809.0   744.0  654.0  656.0   
AMOLATAR                 440.0   544.0  490.0  316.0   

         EPI - BCG to MR 1 dropout rate (%)                     \
                                        Jan   Feb   Mar    Avr   
ABIM                                   22.9  -1.4 -32.2 -22.90   
ADJUMANI                              -18.1 -12.0 -11.8   8.00   
AGAGO                                  12.3   2.2 -34.3 -25.70   
ALEBTONG                                8.9   2.3  -1.4  -4.00   
AMOLATAR                               -5.2   4.2 -19.4  -0.95   

         EPI - DPT 1 to DPT3 dropout rate (%)                    \
                                          Jan   Feb   Mar   Avr   
ABIM                                    -13.1  13.5  17.5  28.0   
ADJUMANI                                 -0.6 -10.8  -2.6  -3.0   
AGAGO                                    -6.3 -13.7  -8.9 -19.8   
ALEBTONG                                -17.6 -14.6  -4.9  -8.7   
AMOLATAR                                 10.8   6.8   7.2  17.6   

         EPI - BCG doses given EPI - BCG to MR 1 dropout rate (%)  \
                       delta_4                            delta_4   
ABIM                 -0.124294                          -2.000000   
ADJUMANI             -0.114754                          -1.441989   
AGAGO                -0.307065                          -3.089431   
ALEBTONG             -0.189122                          -1.449438   
AMOLATAR             -0.281818                          -0.817308   

         EPI - DPT 1 to DPT3 dropout rate (%)  
                                      delta_4  
ABIM                                -3.137405  
ADJUMANI                             4.000000  
AGAGO                                2.142857  
ALEBTONG                            -0.505682  
AMOLATAR                             0.629630

In [114]:
data_target.columns=data_target.columns.map("_".join)

In [117]:
data_target.to_csv(output_path+'epi_deltas.csv')